In [5]:
import pygame
import cv2
import mediapipe as mp
import sys
import random
import time

# Inicialización de Mediapipe
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)

# Inicialización de Pygame
pygame.init()

# Configuración de la ventana del juego
WIDTH, HEIGHT = 1000, 600
GROUND_LEVEL = HEIGHT - 50
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Flappy Head")

# FPS
clock = pygame.time.Clock()
FPS = 30

# Configuración del personaje
bird_width, bird_height = 66, 66
bird_initial_y = HEIGHT // 2 - bird_height // 2  # Posición inicial
bird = pygame.Rect(100, bird_initial_y, bird_width, bird_height)
HEAD_SENSITIVITY = 5  # Ajusta este valor para aumentar la sensibilidad del movimiento

# Vidas
lives = 3
font = pygame.font.SysFont(None, 36)
game_over_font = pygame.font.SysFont(None, 108)

# Puntuación
score = 0
high_score = 0

# Obstáculos
obstacle_width = 50
obstacle_speed = 5
obstacles = []

# Crear obstáculos iniciales
def generate_obstacles():
    global obstacles
    obstacles = []
    for i in range(4):
        height_top = random.randint(100, HEIGHT // 2)
        gap = random.randint(150, 200)
        obstacles.append(pygame.Rect(WIDTH + i * 300, 0, obstacle_width, height_top))
        obstacles.append(pygame.Rect(WIDTH + i * 300, height_top + gap, obstacle_width, HEIGHT - height_top - gap))

# Paisaje
sky_color = (135, 206, 235)
grass_color = (34, 139, 34)
sun_color = (255, 223, 0)
sun_position = (WIDTH - 100, 100)

# Captura de video
cap = cv2.VideoCapture(0)

# Cargar la imagen del avión
avion_image = pygame.image.load('C:/Users/noelp/anaconda3/envs/trabajo_env/avion.png')
avion_image = pygame.transform.scale(avion_image, (bird_width, bird_height))

# Música
audio_file = "musica1.mp3"
pygame.mixer.init()
pygame.mixer.music.load(audio_file)
pygame.mixer.music.play(-1)  # Reproducción en bucle

# Función para reiniciar el juego
def reset_game():
    global bird, paused, score, high_score, obstacle_speed
    bird.y = bird_initial_y
    paused = True
    if score > high_score:
        high_score = score
    score = 0
    obstacle_speed = 5
    generate_obstacles()

# Generar los primeros obstáculos
generate_obstacles()

# Bucle principal
running = True
paused = False
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Capturar fotogramas de la cámara
    ret, frame = cap.read()
    if not ret:
        print("Error al acceder a la cámara.")
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if not paused:
        # Detectar rostro y ajustar la posición del personaje
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                y_center = bboxC.ymin + bboxC.height / 2
                bird_target_y = int(y_center * HEIGHT)
                bird.y += (bird_target_y - bird.y) / HEAD_SENSITIVITY

                # Limitar movimiento dentro de la pantalla
                bird.y = max(0, min(GROUND_LEVEL - bird.height, bird.y))

        # Mover obstáculos y manejar la reaparición
        for obstacle in obstacles[:]:
            obstacle.x -= obstacle_speed

            if obstacle.x + obstacle_width < 0:
                obstacles.remove(obstacle)
                if obstacle.y == 0:  # Solo agregar un nuevo par cuando el obstáculo superior desaparezca
                    height_top = random.randint(100, HEIGHT // 2)
                    gap = random.randint(150, 200)
                    obstacles.append(pygame.Rect(WIDTH, 0, obstacle_width, height_top))
                    obstacles.append(pygame.Rect(WIDTH, height_top + gap, obstacle_width, HEIGHT - height_top - gap))

        # Incrementar puntuación
        score += 1 / FPS

        # Incrementar velocidad cada 7 puntos
        if int(score) % 7 == 0:
            obstacle_speed += 0.1

        # Detección de colisiones
        for obstacle in obstacles:
            if bird.colliderect(obstacle):
                lives -= 1
                if lives > 0:
                    reset_game()
                else:
                    paused = True
                    break

        # Detección de colisión con el suelo
        if bird.y >= GROUND_LEVEL - bird.height:
            lives -= 1
            if lives > 0:
                reset_game()
            else:
                paused = True

    # Dibujar cielo
    screen.fill(sky_color)

    # Dibujar sol
    pygame.draw.circle(screen, sun_color, sun_position, 50)

    # Dibujar césped
    pygame.draw.rect(screen, grass_color, (0, GROUND_LEVEL, WIDTH, HEIGHT - GROUND_LEVEL))

    # Dibujar personaje (con la imagen del avión)
    screen.blit(avion_image, bird.topleft)

    # Dibujar obstáculos
    for obstacle in obstacles:
        pygame.draw.rect(screen, (0, 255, 0), obstacle)

    # Mostrar vidas
    lives_text = font.render(f"Vidas: {lives}", True, (255, 255, 255))
    screen.blit(lives_text, (20, 20))

    # Mostrar puntuación
    score_text = font.render(f"Puntuación: {int(score)}", True, (255, 255, 255))
    screen.blit(score_text, (20, 50))

    # Mostrar récord
    high_score_text = font.render(f"Récord: {int(high_score)}", True, (255, 255, 255))
    screen.blit(high_score_text, (20, 80))

    # Mostrar mensaje de pausa
    if paused:
        if lives == 0:
            game_over_text = game_over_font.render("GAME OVER!", True, (255, 0, 0))
            screen.blit(game_over_text, (WIDTH // 2 - game_over_text.get_width() // 2, HEIGHT // 2 - game_over_text.get_height() // 2))
            pygame.display.flip()
            time.sleep(3)
            running = False
        else:
            pause_text = font.render("Has chocado. Presiona Espacio para volver a comenzar.", True, (255, 255, 255))
            screen.blit(pause_text, (WIDTH // 2 - pause_text.get_width() // 2, HEIGHT // 2))
            keys = pygame.key.get_pressed()
            if keys[pygame.K_SPACE]:
                if lives > 0:
                    paused = False

    # Actualizar pantalla
    pygame.display.flip()

    # Establecer FPS
    clock.tick(FPS)

# Liberar recursos
cap.release()
pygame.quit()
sys.exit()


SystemExit: 